# In-class transcript from Lecture 13, February 20, 2020

# Imports and defs for lecture

In [ ]:
# These are the standard imports for CS 111. 
# This list may change as the quarter goes on.

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline
import cs111

In [ ]:
np.set_printoptions(precision = 4)

In [ ]:
def make_M_from_E(E):
    """Make the (dense) PageRank matrix from the adjacency matrix of a graph
    """
    n = E.shape[0]
    outdegree = np.sum(E,0)
    for j in range(n):
        if outdegree[j] == 0:
            E[:,j] = np.ones(n)
            E[j,j] = 0
    A = E / np.sum(E,0)
    S = np.ones((n,n)) / n
    m = 0.15
    M = (1 - m) * A + m * S
    return M

# Adjacency matrix for page rank example 1

In [ ]:
E1 = np.load('PageRankEG1.npy')
n = E1.shape[0]
print('E1:\n', E1)

# Ranking by in-degree

In [ ]:
indegree = np.sum(E1,1)
indegree

In [ ]:
np.sort(indegree)[::-1]

In [ ]:
degree_perm = np.argsort(indegree)[::-1]
print('indegree ordering:', degree_perm)

In [ ]:
indegree[degree_perm]

# Ranking by eigenvector

In [ ]:
d, V = spla.eig(E1)
print('eigenvalues:', d)

# First fix: Scale columns to sum to one

In [ ]:
print('E1:\n', E1)

In [ ]:
sum(E1,0)

In [ ]:
# Scale columns to sum to one
A = E1 / sum(E1,0)
print('A:\n', A)

In [ ]:
d, V = spla.eig(A)
print('eigenvalues:', d)

In [ ]:
V

In [ ]:
v = V[:,0].real
print('eigenvector v:', v)
print('        A @ v:', A @ v)

In [ ]:
eig_perm = np.argsort(v)[::-1]
print('eigenvector ordering:', eig_perm)

In [ ]:
print('E1:\n', E1)

# Second fix: Handle dangling nodes (vertices with outdegree zero)

In [ ]:
# Create a dangling node (node 2) by erasing the edge (2,0)
E1a = E1.copy()
E1a[0,2] = 0
print('E1a:\n', E1a)

In [ ]:
# Scale columns to sum to one
A = E1a / sum(E1a,0)
print('A:\n', A)

In [ ]:
d, V = spla.eig(A)
print('eigenvalues:', d)

In [ ]:
print('E1a:\n', E1a)

# Let the dangling node point to everyone


In [ ]:
E1a[:,2] = np.ones(n)
E1a[2,2] = 0
print('E1a:\n', E1a)

In [ ]:
# Scale columns to sum to one
A = E1a / sum(E1a,0)
print('A:\n', A)

In [ ]:
d, V = spla.eig(A)
print('eigenvalues:', d)

In [ ]:
v = V[:,0].real
print('eigenvector v:', v)
print('        A @ v:', A @ v)
eig_perm = np.argsort(v)[::-1]
print('eigenvector ordering:', eig_perm)

# Third fix: Make the matrix strongly connected

In [ ]:
E2 = np.load('PageRankEG2.npy')
print('E2:\n', E2)
n = E2.shape[0]

In [ ]:
A = E2 / sum(E2,0)
print('A:\n', A)

In [ ]:
d, V = spla.eig(A)
print('eigenvalues:', d)

In [ ]:
print('V:\n', V)

In [ ]:
# put a small fixed weight m from each node to every other node
m = .15
print(m * np.ones((n,n))/n)

In [ ]:
# Now get the final matrix as a mixture of A and the small fixed weights
M = (1-m) * A + m * np.ones((n,n))/n
print('M:\n', M)

In [ ]:
# check that the columns still sum to one
print(np.sum(M,0))

In [ ]:
d, V = spla.eig(M)
print('eigenvalues:', d)

In [ ]:
make_M_from_E(E2)

In [ ]:
i = np.argmax(d)
print('largest eigenvalue is at position', i, ' with value', d[i])
v = V[:,i].real
print('eigenvector v:', v)
print('        M @ v:', M @ v)
eig_perm = np.argsort(v)[::-1]
print('eigenvector ordering:', eig_perm)

# The power method to compute the eigenvector with the largest-magnitude eigenvalue

In [ ]:
# We'll go back to the first graph
E1 = np.load('PageRankEG1.npy')
n = E1.shape[0]
print('E1:\n', E1)

In [ ]:
M = make_M_from_E(E1)
print('M:\n', M)
print()
d, V = spla.eig(M)
print('eigenvalues:', d)
i = np.argmax(d)
print('largest eigenvalue is at position', i, ' with value', d[i])
print()
v = V[:,i].real
print('eigenvector v:', v)
print('        M @ v:', M @ v)
eig_perm = np.argsort(v)[::-1]
print()
print('eigenvector ordering:', eig_perm)

In [ ]:
# Power method
x = np.ones(n)/n
print(0, x)
for i in range(10):
    x = M @ x
    x = x / npla.norm(x)
    print(i+1, x)

In [ ]:
v

In [ ]:
r = cs111.pagerank1(E1)
print('ranking from cs111.pagerank1:', r)

# A larger example: The Harvard web crawl (from long ago)

In [ ]:
E3 = np.load('PageRankEG3.npy')
E3.shape

In [ ]:
%matplotlib inline
plt.spy(E3)

In [ ]:
with open('PageRankEG3.nodelabels') as f:
    labels = f.read().splitlines()

In [ ]:
for i in range(10):
    print(i, labels[i])

In [ ]:
M = make_M_from_E(E3)
M.shape

In [ ]:
np.sum(np.sum(E3))

In [ ]:
d, V = spla.eig(M)
i = np.argmax(d)
print('largest eigenvalue is at position', i, ' with value', d[i])

In [ ]:
v = V[:,i].real
#print('eigenvector v:', v)
#print('        M @ v:', M @ v)
eig_perm = np.argsort(v)[::-1]
print('eigenvector ordering:', eig_perm[:10], ' ...')

In [ ]:
for i in range(10):
    print(i, eig_perm[i], labels[eig_perm[i]])

# A much bigger web crawl

In [ ]:
E = scipy.sparse.load_npz('webGoogle.npz')
E.shape

If we tried make_M_from_E() on this matrix, Jupyter would run out of memory, since M has nearly a trillion nonzero elements. And even if we could store M, it would take a vastly long time to run either npla.eig() (which is an $O(n^3)$ algorithm) or the power method (which would be $O(n^2)$ per iteration on M).

The goal of Homework 6 is to implement code that runs the power method on M, but without forming M explicitly. It turns out that, if you're crafty, you can obtain the effect of multiplying M by a vector without ever forming any matrices besides E.

Scipy does have a sparse-matrix eigensolver. We can't use it on M because M has too many nonzeros. But, just to get a feeling for what it can do, we can use scipy.sparse.linalg.eigs() to find a few eigenvalues of the original adjacency matrix E. Those don't help us with PageRank, but it's interesting to see how long it takes anyway.

In [ ]:
%time d, V = scipy.sparse.linalg.eigs(E) 

In [ ]:
print('some eigenvalues of E (not of M):\n', d)